In [8]:
# === test1 (modified to use PHYSICAL-CENTERED FOV) ===
import os
import SimpleITK as sitk
import numpy as np

image_dir = "./MRI/output"
mask_dir = "./MRI/seg_out"
output_image_dir = "./MRI/output_224/images"
output_mask_dir = "./MRI/output_224/masks"
cache_dir = "./MRI/n4_cache"
exclude_files = []

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)

# ⚙️ Target spacing & FOV (mm)
TARGET_SPACING = (1.0, 1.0, 1.0)                  # (x,y,z) mm
TARGET_PHYSICAL_SIZE_MM = (224.0, 224.0, 128.0)   # (x,y,z) mm ➜ out size = (224,224,128) voxels

# ✅ Bias field correction with caching (Otsu body mask)
def bias_field_correction_cached(image, fname):
    cached_path = os.path.join(cache_dir, fname)
    if os.path.exists(cached_path):
        print(f"⚡ Using cached bias corrected: {fname}")
        return sitk.ReadImage(cached_path)

    image = sitk.Cast(image, sitk.sitkFloat32)
    mask = sitk.OtsuThreshold(image, 0, 1, 200)
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected = corrector.Execute(image, mask)
    sitk.WriteImage(corrected, cached_path)
    print(f"✅ Cached bias corrected: {fname}")
    return corrected

# ✅ Z-score normalization (>0 voxels)
def normalize_image(itk_image):
    image_np = sitk.GetArrayFromImage(itk_image).astype(np.float32)
    nonzero = image_np[image_np > 0]
    if nonzero.size == 0:
        return itk_image
    mean, std = np.mean(nonzero), np.std(nonzero)
    normalized = (image_np - mean) / (std + 1e-8)
    normalized[image_np == 0] = 0
    norm_image = sitk.GetImageFromArray(normalized.astype(np.float32))
    norm_image.CopyInformation(itk_image)
    return norm_image

# ✅ PHYSICAL-CENTERED FOV resampling (no pad/crop)
def resample_to_physical_fov_centered(
    itk_image,
    target_spacing=(1.0, 1.0, 1.0),
    target_physical_size_mm=(224.0, 224.0, 128.0),
    is_label=False,
    orient_to="RAI",
):
    """
    - Optionally orient to RAI to avoid direction issues.
    - Compute physical center of input.
    - Place a fixed-mm FOV (target_physical_size_mm) centered at that point.
    - Output size = round(FOV_mm / spacing_mm).
    """
    img = itk_image
    if orient_to is not None:
        img = sitk.DICOMOrient(img, orient_to)

    # Output voxel size from FOV(mm) / spacing(mm)
    out_size_xyz = [int(round(ps / sp)) for ps, sp in zip(target_physical_size_mm, target_spacing)]
    out_space = np.array(target_spacing, dtype=float)

    # Physical center of input image
    size_xyz = np.array(img.GetSize(), dtype=float)
    center_idx = 0.5 * (size_xyz - 1.0)
    center_phys = np.array(img.TransformContinuousIndexToPhysicalPoint(center_idx.tolist()))
    R = np.array(img.GetDirection()).reshape(3, 3)

    # Half-extent of output FOV in mm (use (out_size-1)*spacing/2 to align voxel centers)
    half_out = ((np.array(out_size_xyz, dtype=float) - 1.0) * out_space) / 2.0

    # Compute output origin so that the FOV is centered at center_phys
    origin_out = center_phys - R @ half_out

    # Resample
    res = sitk.ResampleImageFilter()
    res.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)
    res.SetOutputSpacing(tuple(target_spacing))
    res.SetSize([int(v) for v in out_size_xyz])
    res.SetOutputDirection(tuple(img.GetDirection()))
    res.SetOutputOrigin(tuple(origin_out))
    res.SetDefaultPixelValue(0)
    return res.Execute(img)

# 🔁 Main loop
for fname in sorted(os.listdir(image_dir)):
    if not fname.endswith(".nii.gz") or fname in exclude_files:
        continue

    image_path = os.path.join(image_dir, fname)
    mask_path  = os.path.join(mask_dir,  fname)

    if not os.path.exists(mask_path):
        print(f"❌ Mask not found for {fname}")
        continue

    try:
        print(f"🔧 Processing: {fname}")

        # 1) Load
        image = sitk.ReadImage(image_path)
        mask  = sitk.ReadImage(mask_path)

        # 2) N4 (cached)
        image = bias_field_correction_cached(image, fname)

        # 3) Normalization
        image = normalize_image(image)

        # 4) Physical-centered FOV resample (no pad/crop)
        image_out = resample_to_physical_fov_centered(
            image,
            target_spacing=TARGET_SPACING,
            target_physical_size_mm=TARGET_PHYSICAL_SIZE_MM,
            is_label=False,
            orient_to="RAI",
        )
        mask_out = resample_to_physical_fov_centered(
            mask,
            target_spacing=TARGET_SPACING,
            target_physical_size_mm=TARGET_PHYSICAL_SIZE_MM,
            is_label=True,
            orient_to="RAI",
        )

        # 5) Save
        sitk.WriteImage(image_out, os.path.join(output_image_dir, fname))
        sitk.WriteImage(mask_out,  os.path.join(output_mask_dir,  fname))
        print(f"✅ Done: {fname}")

    except Exception as e:
        print(f"❌ Failed {fname}: {e}")


🔧 Processing: CCTH-A07_0000.nii.gz
⚡ Using cached bias corrected: CCTH-A07_0000.nii.gz
✅ Done: CCTH-A07_0000.nii.gz
🔧 Processing: CCTH-A08_0000.nii.gz
⚡ Using cached bias corrected: CCTH-A08_0000.nii.gz
✅ Done: CCTH-A08_0000.nii.gz
🔧 Processing: TCGA-VS-A8EB_0000.nii.gz
⚡ Using cached bias corrected: TCGA-VS-A8EB_0000.nii.gz
✅ Done: TCGA-VS-A8EB_0000.nii.gz
🔧 Processing: TCGA-VS-A8EC_0000.nii.gz
⚡ Using cached bias corrected: TCGA-VS-A8EC_0000.nii.gz
✅ Done: TCGA-VS-A8EC_0000.nii.gz
🔧 Processing: TCGA-VS-A8EG_0000.nii.gz
⚡ Using cached bias corrected: TCGA-VS-A8EG_0000.nii.gz
✅ Done: TCGA-VS-A8EG_0000.nii.gz
🔧 Processing: TCGA-VS-A8EH_0000.nii.gz
⚡ Using cached bias corrected: TCGA-VS-A8EH_0000.nii.gz
✅ Done: TCGA-VS-A8EH_0000.nii.gz
🔧 Processing: TCGA-VS-A8EI_0000.nii.gz
⚡ Using cached bias corrected: TCGA-VS-A8EI_0000.nii.gz
✅ Done: TCGA-VS-A8EI_0000.nii.gz
🔧 Processing: TCGA-VS-A8EJ_0000.nii.gz
⚡ Using cached bias corrected: TCGA-VS-A8EJ_0000.nii.gz
✅ Done: TCGA-VS-A8EJ_0000.nii.gz
